In [18]:
from pycox.evaluation import EvalSurv
import torch
import torchtuples as tt
import os
import pickle
from lifelines.utils import concordance_index

from utils import dataset_name
from eval_utils import MODEL_CLASS, get_results, DL_MODELS, ML_MODELS, MODELS
import numpy as np
import pandas as pd
np.random.seed(42)
_ = torch.manual_seed(42)


print(f"Experiments over {dataset_name}")

Experiments over prostateSurvival


In [19]:
train = pd.read_csv(f"../../datasets/train/{dataset_name}.csv")
test = pd.read_csv(f"../../datasets/test/{dataset_name}.csv")

X, y = train.drop(["event", "duration"], axis=1), train[["event", "duration"]]
X_test, y_test = test.drop(["event", "duration"], axis=1), test[["event", "duration"]]

# Finding the best parameters for each model and run the models over the test data

In [20]:
results = pd.DataFrame()
     
for model in MODELS:
    stats = pickle.load( open( f"statistics/{model}/best_model.pkl", "rb" ))
    df = pd.DataFrame(stats, index=[model])
    results = pd.concat([results, df], axis=0)
    
c_index_df = results[["c_index", "c_index_std", "c_index_params"]].sort_values("c_index", ascending=False)
concordance_td_df = results[["concordance_td", "concordance_td_std", "concordance_td_params"]].sort_values("concordance_td", ascending=False)
ibs_df = results[["ibs", "ibs_std", "ibs_params"]].sort_values("ibs", ascending=True)

## `c index`

In [21]:
c_index_df

,c_index,c_index_std,c_index_params
deep_surv,0.782084,0.018315,"deep_surv_dropout_0.1_num_nodes_[28, 28, 100, ..."
gbst,0.779542,0.018669,gbst_learning_rate_0.1_n_estimators_100_subsam...
rsf,0.778963,0.018635,rsf_n_estimators_100_max_depth_3_min_samples_s...
cox_time,0.774480,0.018246,"cox_time_dropout_0.1_num_nodes_[28, 28, 100, 2..."
reg_coxph,0.769764,0.020978,reg_coxph_l1_ratio_0.5_tol_0.001_max_iter_1000...
pc_hazard,0.623830,0.152788,"pc_hazard_dropout_0.1_num_nodes_[16, 16, 16, 6..."
deep_hit,0.299190,0.102216,"deep_hit_dropout_0.1_num_nodes_[3, 5]_activati..."


In [22]:
results = []
column = "c_index_params"


for model_name, params in zip(c_index_df.index, c_index_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, np.round(val,4))

deep_hit 0.7836
reg_coxph 0.7835
rsf 0.777
gbst 0.7765
deep_surv 0.7757
cox_time 0.7743
pc_hazard 0.2673


___

## `concordance td`

In [23]:
concordance_td_df

,concordance_td,concordance_td_std,concordance_td_params
deep_surv,0.760880,0.018237,deep_surv_dropout_0.16_num_nodes_[41]_activati...
gbst,0.754257,0.018844,gbst_learning_rate_0.1_n_estimators_100_subsam...
deep_hit,0.751363,0.017351,"deep_hit_dropout_0.3_num_nodes_[28, 28, 100, 2..."
cox_time,0.750433,0.016507,"cox_time_dropout_0.1_num_nodes_[32, 128, 128]_..."
reg_coxph,0.744480,0.020870,reg_coxph_l1_ratio_0.5_tol_0.001_max_iter_1000...
rsf,0.744401,0.022744,rsf_n_estimators_200_max_depth_3_min_samples_s...
pc_hazard,0.317471,0.025308,pc_hazard_dropout_0.3_num_nodes_[100]_activati...


In [24]:
results = []
column = "concordance_td_params"


for model_name, params in zip(concordance_td_df.index, concordance_td_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, np.round(val,4))

deep_surv 0.766
deep_hit 0.7603
reg_coxph 0.7586
rsf 0.758
cox_time 0.7571
gbst 0.7516
pc_hazard 0.6062


## `integrated brier score`

In [25]:
ibs_df

,ibs,ibs_std,ibs_params
pc_hazard,0.009459,0.011183,"pc_hazard_dropout_0.1_num_nodes_[16, 16, 16, 6..."
deep_surv,0.055713,0.002205,"deep_surv_dropout_0.1_num_nodes_[32, 128, 128]..."
rsf,0.056924,0.002166,rsf_n_estimators_200_max_depth_5_min_samples_s...
cox_time,0.057751,0.002197,"cox_time_dropout_0.2_num_nodes_[16, 32, 64, 64..."
gbst,0.057862,0.001777,gbst_learning_rate_0.1_n_estimators_100_subsam...
reg_coxph,0.058417,0.002507,reg_coxph_l1_ratio_0.1_tol_1e-07_max_iter_1000...
deep_hit,0.289333,0.009895,"deep_hit_dropout_0.1_num_nodes_[16, 32, 64, 64..."


In [26]:
results = []
column = "ibs_params"


for model_name, params in zip(ibs_df.index, ibs_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))

for m,val in sorted(results, key=lambda x: x[1], reverse=False):
    print(m, np.round(val,4))

reg_coxph 0.0586
cox_time 0.0587
deep_surv 0.0602
rsf 0.061
gbst 0.0615
deep_hit 0.077
pc_hazard 0.5331
